In [469]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score, accuracy_score, mean_squared_error
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression, Ridge

In [470]:
data = 'https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv'

#!wget $data

In [471]:
housing = pd.read_csv('/home/frog/Documents/Coding/MLZoomCamp-1/Week_3/housing.csv')
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [472]:
housing = housing.fillna(housing.median())

housing.isna().sum()

/tmp/ipykernel_3610/2391650053.py:1: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  housing = housing.fillna(housing.median())


longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
median_house_value    0
ocean_proximity       0
dtype: int64

In [473]:
housing['rooms_per_household'] = housing['total_rooms'] / housing['households']
housing['bedrooms_per_room'] = housing['total_bedrooms'] / housing['total_rooms']
housing['population_per_household'] = housing['population'] / housing['households']

housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,rooms_per_household,bedrooms_per_room,population_per_household
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY,6.984127,0.146591,2.555556
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY,6.238137,0.155797,2.109842
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY,8.288136,0.129516,2.802260
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY,5.817352,0.184458,2.547945
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY,6.281853,0.172096,2.181467


In [474]:
housing['ocean_proximity'].value_counts()
# or housing['ocean_proximity'].mode()

<1H OCEAN     9136
INLAND        6551
NEAR OCEAN    2658
NEAR BAY      2290
ISLAND           5
Name: ocean_proximity, dtype: int64

Question 1: <1H OCEAN.

In [475]:
housing.dtypes

longitude                   float64
latitude                    float64
housing_median_age          float64
total_rooms                 float64
total_bedrooms              float64
population                  float64
households                  float64
median_income               float64
median_house_value          float64
ocean_proximity              object
rooms_per_household         float64
bedrooms_per_room           float64
population_per_household    float64
dtype: object

In [476]:
housing.columns

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value', 'ocean_proximity', 'rooms_per_household',
       'bedrooms_per_room', 'population_per_household'],
      dtype='object')

In [477]:
numerical = ['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'rooms_per_household', 'bedrooms_per_room', 'population_per_household']
categorical = ['ocean_proximity']

In [478]:
corr_mat = housing[numerical].corr().abs()
upper_corr_mat = corr_mat.where(np.triu(np.ones(corr_mat.shape), k=1).astype(bool))
unique_corr_pairs = upper_corr_mat.unstack().dropna()
unique_corr_pairs.sort_values(ascending=False)

households                total_bedrooms         0.974366
total_bedrooms            total_rooms            0.927058
latitude                  longitude              0.924664
households                total_rooms            0.918484
                          population             0.907222
population                total_bedrooms         0.873535
                          total_rooms            0.857126
bedrooms_per_room         median_income          0.545298
                          rooms_per_household    0.370308
total_rooms               housing_median_age     0.361262
rooms_per_household       median_income          0.326895
total_bedrooms            housing_median_age     0.319026
households                housing_median_age     0.302916
population                housing_median_age     0.296244
median_income             total_rooms            0.198050
bedrooms_per_room         total_rooms            0.187381
rooms_per_household       housing_median_age     0.153277
bedrooms_per_r

Question 2: total_bedrooms and households.

In [479]:
mean = housing['median_house_value'].mean()

housing['above_average'] = np.where(housing['median_house_value'] >= mean, 1, 0)

housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,rooms_per_household,bedrooms_per_room,population_per_household,above_average
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY,6.984127,0.146591,2.555556,1
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY,6.238137,0.155797,2.109842,1
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY,8.288136,0.129516,2.802260,1
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY,5.817352,0.184458,2.547945,1
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY,6.281853,0.172096,2.181467,1


In [480]:
housing_full_train, housing_test = train_test_split(housing, test_size=0.2, random_state=42)
housing_train, housing_val = train_test_split(housing_full_train, test_size=0.25, random_state=42)

len(housing_train), len(housing_val), len(housing_test)

(12384, 4128, 4128)

In [481]:
housing_train = housing_train.reset_index(drop=True)
housing_val = housing_val.reset_index(drop=True)
housing_test = housing_test.reset_index(drop=True)

y_train = housing_train['above_average'].values
y_val = housing_val['above_average'].values
y_test = housing_test['above_average'].values

del housing_train['above_average']
del housing_val['above_average']
del housing_test['above_average']

In [482]:
round(mutual_info_score(housing_train['ocean_proximity'], y_train),2)

0.1

Question 3: 0.10.

One hot encoding the target variable, ocean proximity.

In [483]:
dv = DictVectorizer(sparse=False)

train_dict = housing_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = housing_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [484]:
model = LogisticRegression(solver="liblinear", C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [485]:
y_pred = model.predict(X_val)

In [486]:
accuracy = np.round(accuracy_score(y_val, y_pred),2)
print(accuracy)

0.84


Question 4: 0.95 ?? This just seems way too high, probably something is wrong :(

This was indeed wrong, something is wrong with the data. 

After 1 hour debugging, I found out I included the target variable in the numerical variable that contained the numerical columns :)
The correct solution is 0.84.

In [487]:
small =  ['total_rooms', 'total_bedrooms', 'population', 'households', 'above_average']
small_households = ['total_rooms', 'total_bedrooms', 'population', 'above_average']
small_population = ['total_rooms', 'total_bedrooms',  'households', 'above_average']
small_total_bedrooms =  ['total_rooms', 'population', 'households', 'above_average']
small_total_rooms =  ['total_bedrooms', 'population', 'households', 'above_average']

small_array = {'households': small_households, 'population': small_population, 'total_bedrooms': small_total_bedrooms, 'small_total_rooms': small_total_rooms}

In [488]:
def calc_log_reg(columns):
    housing_full_train, housing_test = train_test_split(housing[columns], test_size=0.2, random_state=42)
    housing_train, housing_val = train_test_split(housing_full_train, test_size=0.25, random_state=42)

    housing_train = housing_train.reset_index(drop=True)
    housing_val = housing_val.reset_index(drop=True)
    housing_test = housing_test.reset_index(drop=True)

    y_train = housing_train['above_average'].values
    y_val = housing_val['above_average'].values
    y_test = housing_test['above_average'].values

    del housing_train['above_average']
    del housing_val['above_average']
    del housing_test['above_average']

    model = LogisticRegression(solver="liblinear", C=1.0, max_iter=1000, random_state=42)
    model.fit(housing_train, y_train)

    y_pred = model.predict(housing_val)

    accuracy = np.round(accuracy_score(y_val, y_pred),2)
    return(accuracy)

In [489]:
benchmark_acc = calc_log_reg(small)

In [490]:
for key, value in small_array.items():
    print(key, round(benchmark_acc - calc_log_reg(value),2))

households 0.04
population 0.05
total_bedrooms 0.05
small_total_rooms 0.08


Question 5: the smallest difference is the group in which 'households' is not included in (value of 0.04).

In [491]:
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,rooms_per_household,bedrooms_per_room,population_per_household,above_average
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY,6.984127,0.146591,2.555556,1
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY,6.238137,0.155797,2.109842,1
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY,8.288136,0.129516,2.802260,1
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY,5.817352,0.184458,2.547945,1
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY,6.281853,0.172096,2.181467,1


In [492]:
housing_full_train, housing_test = train_test_split(housing, test_size=0.2, random_state=42)
housing_train, housing_val = train_test_split(housing_full_train, test_size=0.25, random_state=42)

housing_train = housing_train.reset_index(drop=True)
housing_val = housing_val.reset_index(drop=True)
housing_test = housing_test.reset_index(drop=True)

y_train = np.log1p(housing_train['median_house_value'].values)
y_val = np.log1p(housing_val['median_house_value'].values)
y_test = np.log1p(housing_test['median_house_value'].values)

del housing_train['median_house_value']
del housing_val['median_house_value']
del housing_test['median_house_value']

In [493]:
housing.columns

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value', 'ocean_proximity', 'rooms_per_household',
       'bedrooms_per_room', 'population_per_household', 'above_average'],
      dtype='object')

In [494]:
dv = DictVectorizer(sparse=False)

train_dict = housing_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

train_val = housing_val[categorical + numerical].to_dict(orient='records')
X_val = dv.fit_transform(val_dict)

In [495]:
for a in [0, 0.01, 0.1, 1, 10]:
    model = Ridge(alpha=a, solver="sag", random_state=42)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_val)

    print(a, np.round(mean_squared_error(y_val, y_pred),4))

0 0.2746
0.01 0.2746
0.1 0.2746
1 0.2746
10 0.2746


Question 6: RMSE looks exactly the same for each alpha (~0.275). As such, we choose the smaller one, alpha = 0.